In [8]:
# Import libraries

import gw3RACE_functions as gw
import pandas as pd

%matplotlib inline

In [10]:
# Example code
df = pd.read_csv('output_Spombe/output_short.tab', sep = '\t',
                 names = ['read_ID', 'chr', 'start_R1', 'stop_R1','strand_R1', 'gene_start', 'gene_stop',
                         'gene','coord_R2', 'cigar', 'seq_R2'])

df['RNA_type'] = df['gene'].apply(lambda x: gw.test_RNA_type(x.lower()))
df['tail_fromcigar'] = df.apply(lambda kol: gw.take_tail_fromcigar8(kol.strand_R1, kol.cigar,  kol.seq_R2), 
                                axis = 1)
df['tail_LENcigar'] = df['tail_fromcigar'].apply(lambda x: len(x))
df['tail_fromGREP'] = df.apply(lambda kol: gw.grep_tail_edit_onlyfromSeq(kol.seq_R2), axis = 1)

#df = df.drop_duplicates()
df['tail_GreporCigar'] = df.apply(lambda kol: gw.tail_fromGREPorCIGAR(kol.cigar, kol.tail_fromGREP, kol.tail_fromcigar),axis = 1)


                                                                                    
df['tail_from'] = df.apply(lambda kol: gw.tail_fromGREPorCIGAR_description(kol.cigar,
                                                                        kol.tail_fromGREP,kol.tail_fromcigar),
                           axis = 1)
df['tail_len'] = df['tail_GreporCigar'].apply(lambda x: len(x))
#df = df[df['tail_len']>0]
df['tail_type'] = df.apply(lambda kol:   gw.test_tail_cigargrep8(kol.tail_from, kol.strand_R1, kol.tail_GreporCigar),axis = 1)
  

df['stop_R2'] =  df.apply(lambda kol: gw.stop_based_on_cigar(kol.cigar, kol.strand_R1 , kol.coord_R2),axis = 1)
df['distance_to_TES'] =  df.apply(lambda kol: gw.distance_to_TES(kol.cigar,kol.strand_R1 ,kol.stop_R2,
                                                              kol.gene_start,kol.gene_stop),axis = 1)

df['gene_len'] = df['gene_stop']-df['gene_start']
df['rel_distance_to_TES'] = df['distance_to_TES']/df['gene_len']
df.to_csv('output_Spombe/output_detailed_DataFrame.csv')
    

In [18]:
df.head()

,read_ID,chr,start_R1,stop_R1,strand_R1,gene_start,gene_stop,gene,coord_R2,cigar,...,tail_LENcigar,tail_fromGREP,tail_GreporCigar,tail_from,tail_len,tail_type,stop_R2,distance_to_TES,gene_len,rel_distance_to_TES
0,A00805:163:HFM2WDRX2:1:1101:1000:1626:ANTGTT,III,139440,139539,-,138958,140937,SPCC320.14,139441,90M,...,0,TTTT,,cigar,0,no_tail,139440,-482,1979,-0.243557
1,A00805:163:HFM2WDRX2:1:1101:1000:19445:CNCCTG,II,677512,677613,-,676632,679966,SPBC947.02,677362,90M,...,0,A,,cigar,0,no_tail,677361,-729,3334,-0.218656
2,A00805:163:HFM2WDRX2:1:1101:1000:27085:CTTTAT,II,3958854,3958955,-,3958831,3959086,SPSNRNA.07,3958833,90M,...,0,A,,cigar,0,no_tail,3958832,-1,255,-0.003922
3,A00805:163:HFM2WDRX2:1:1101:10004:28682:TCACAC,II,4377200,4377301,-,4377220,4379500,SPBC1539.09c,4377091,14S76M,...,14,TTTTTTTTTTTTTTT,TTTTTTTTTTTTTT,cigar,14,polyA,4377090,130,2280,0.057018
4,A00805:163:HFM2WDRX2:1:1101:10004:7075:TCTTCA,II,3958860,3958961,-,3958831,3959086,SPSNRNA.07,3958833,90M,...,0,A,,cigar,0,no_tail,3958832,-1,255,-0.003922
